# **PDAC CellTracksColab — Track Clustering & Co-Arrest**
---

<font size=4>This notebook is part of the **CellTracksColab** suite for analyzing cancer cell–endothelium interactions in flow. It provides two complementary, lightweight modules you can run end-to-end on your tracked data.

**Project:** [CellMigrationLab/CellTracksColab](https://github.com/CellMigrationLab/CellTracksColab)

---

### 1) Spatial Clustering (Ripley’s L)
- **Goal:** Test whether arrested cells are spatially clustered within each FOV.
- **Input:** Arrest points per FOV.
- **Output:** L(r) curves, per-FOV and grouped summaries by metadata (Cells / Treatment / Condition), and compact plots.

### 2) Space–Time Co-Arrest
- **Goal:** Quantify arrests that occur **together** in space and time and characterize **cluster size**.
- **Definition:** For each track, take the first frame where speed ≤ threshold as the arrest event. An event is “together” if ≥1 other event lies within a space–time neighborhood (**ΔT**, **r**), with **r = k × diameter(Cells)** to normalize across cell sizes.
- **Output:** Co-arrest fraction per FOV, event-level cluster sizes (size≥2), distributions by Cells / Treatment / Condition / Cells×Treatment, time-colored arrest maps, and QC (k–ΔT sensitivity; flag vs. component agreement).

---


**Notebook creation:** [Guillaume Jacquemet](https://cellmig.org/)


In [ ]:
# @title #MIT License

print("""
**MIT License**

Copyright (c) 2023 Guillaume Jacquemet

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.""")

--------------------------------------------------------
# **Part 1: Prepare the session and load your data**
--------------------------------------------------------


## **1.1. Install key dependencies**
---
<font size = 4>

In [ ]:
#@markdown ##Play to install
!pip -q install pandas scikit-learn
!pip -q install hdbscan
!pip -q install umap-learn
!pip -q install plotly
!pip -q install tqdm

!git clone https://github.com/CellMigrationLab/CellTracksColab.git


import ipywidgets as widgets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import itertools
from matplotlib.gridspec import GridSpec
import requests

import os
import pandas as pd
import seaborn as sns
import numpy as np
import sys
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import itertools
import requests
import ipywidgets as widgets
import warnings
import scipy.stats as stats

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec
from ipywidgets import Dropdown, interact,Layout, VBox, Button, Accordion, SelectMultiple, IntText
from tqdm.notebook import tqdm
from IPython.display import display, clear_output
from scipy.spatial import ConvexHull
from scipy.spatial.distance import cosine, pdist
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.metrics import pairwise_distances
from scipy.stats import zscore, ks_2samp
from sklearn.preprocessing import MinMaxScaler
from multiprocessing import Pool
from matplotlib.ticker import FixedLocator
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import LogNorm
sys.path.append("../")
sys.path.append("CellTracksColab/")

import celltracks
from celltracks import *
from celltracks.Track_Plots import *
from celltracks.BoxPlots_Statistics import *
from celltracks.Track_Metrics import *


def save_dataframe_with_progress(df, path, desc="Saving", chunk_size=500000):
    """Save a DataFrame with a progress bar and gzip compression."""

    # Estimating the number of chunks based on the provided chunk size
    num_chunks = int(len(df) / chunk_size) + 1

    # Create a tqdm instance for progress tracking
    with tqdm(total=len(df), unit="rows", desc=desc) as pbar:
        # Open the file for writing with gzip compression
        with gzip.open(path, "wt") as f:
            # Write the header once at the beginning
            df.head(0).to_csv(f, index=False)

            for chunk in np.array_split(df, num_chunks):
                chunk.to_csv(f, mode="a", header=False, index=False)
                pbar.update(len(chunk))

## **1.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the instructions.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

from google.colab import drive
drive.mount('/content/gdrive/')



## **1.3. Compile your data or load existing dataframes**
---



In [ ]:
#@markdown ##Provide the path to the dataset:


import os
import re
import glob
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import requests
import zipfile

#@markdown ###You have existing dataframes, provide the path to your:

Track_table = ''  # @param {type: "string"}
Spot_table = ''  # @param {type: "string"}

#@markdown ###Provide the path to your Result folder

Results_Folder = ""  # @param {type: "string"}

if not Results_Folder:
    Results_Folder = '/content/Results'  # Default Results_Folder path if not defined

if not os.path.exists(Results_Folder):
    os.makedirs(Results_Folder)  # Create Results_Folder if it doesn't exist

# Print the location of the result folder
print(f"Result folder is located at: {Results_Folder}")

def validate_tracks_df(df):
    """Validate the tracks dataframe for necessary columns and data types."""
    required_columns = ['TRACK_ID']
    for col in required_columns:
        if col not in df.columns:
            print(f"Error: Column '{col}' missing in tracks dataframe.")
            return False

    # Additional data type checks or value ranges can be added here
    return True

def validate_spots_df(df):
    """Validate the spots dataframe for necessary columns and data types."""
    required_columns = ['TRACK_ID', 'POSITION_X', 'POSITION_Y', 'POSITION_T']
    for col in required_columns:
        if col not in df.columns:
            print(f"Error: Column '{col}' missing in spots dataframe.")
            return False

    # Additional data type checks or value ranges can be added here
    return True

def check_unique_id_match(df1, df2):
    df1_ids = set(df1['Unique_ID'])
    df2_ids = set(df2['Unique_ID'])

    # Check if the IDs in the two dataframes match
    if df1_ids == df2_ids:
        print("The Unique_ID values in both dataframes match perfectly!")
    else:
        missing_in_df1 = df2_ids - df1_ids
        missing_in_df2 = df1_ids - df2_ids

        if missing_in_df1:
            print(f"There are {len(missing_in_df1)} Unique_ID values present in the second dataframe but missing in the first.")
            print("Examples of these IDs are:", list(missing_in_df1)[:5])

        if missing_in_df2:
            print(f"There are {len(missing_in_df2)} Unique_ID values present in the first dataframe but missing in the second.")
            print("Examples of these IDs are:", list(missing_in_df2)[:5])

# For existing dataframes
if Track_table:
    print("Loading track table file....")
    merged_tracks_df = pd.read_csv(Track_table, low_memory=False)
    if not validate_tracks_df(merged_tracks_df):
        print("Error: Validation failed for loaded tracks dataframe.")

if Spot_table:
    print("Loading spot table file....")
    merged_spots_df = pd.read_csv(Spot_table, low_memory=False)
    if not validate_spots_df(merged_spots_df):
        print("Error: Validation failed for loaded spots dataframe.")

check_for_nans(merged_spots_df, "merged_spots_df")
check_for_nans(merged_tracks_df, "merged_tracks_df")


In [ ]:
#@markdown ##Check Metadata


# Define the metadata columns that are expected to have identical values for each filename
metadata_columns = ['Cells', 'Flow_speed', 'Treatment', 'Condition', 'experiment_nb', 'Repeat']

# Group the DataFrame by 'File_name' and then check if all entries within each group are identical
consistent_metadata = True
for name, group in merged_tracks_df.groupby('File_name'):
    for col in metadata_columns:
        if not group[col].nunique() == 1:
            consistent_metadata = False
            print(f"Inconsistency found for file: {name} in column: {col}")
            break  # Stop checking other columns for this group and move to the next file
    if not consistent_metadata:
        break  # Stop the entire process if any inconsistency is found

if consistent_metadata:
    print("All files have consistent metadata across the specified columns.")
else:
    print("There are inconsistencies in the metadata. Please check the output for details.")

# Drop duplicates based on the 'File_name' to get a unique list of filenames and their metadata
unique_files_df = merged_tracks_df.drop_duplicates(subset=['File_name'])[['File_name', 'Cells', 'Flow_speed', 'Treatment', 'Condition', 'experiment_nb', 'Repeat']]

# Reset the index to clean up the DataFrame
unique_files_df.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame in a nicely formatted HTML table
unique_files_df

import pandas as pd

# Assuming 'df' is your DataFrame and it already contains 'Conditions' and 'Repeats' columns.

# Group by 'Conditions' and 'Repeats' and count the occurrences
grouped = unique_files_df.groupby(['Condition', 'Repeat']).size().reset_index(name='counts')

# Check if any combinations have a count greater than 1, which means they are not unique
non_unique_combinations = grouped[grouped['counts'] > 1]

# Print the non-unique combinations
if not non_unique_combinations.empty:
    print("There are non-unique combinations of Conditions and Repeats:")
    print(non_unique_combinations)
else:
    print("All combinations of Conditions and Repeats are unique.")

check_unique_id_match(merged_spots_df, merged_tracks_df)


# Group the DataFrame by 'Cells', 'ILbeta', 'Repeat' and then check if there are 4 unique 'Flow_speed' values for each group
consistent_flow_speeds = True
for (cells, ilbeta, repeat), group in merged_tracks_df.groupby(['Cells', 'Treatment', 'Repeat']):
    if group['Flow_speed'].nunique() != 4:
        consistent_flow_speeds = False
        print(f"Inconsistency found for Cells: {cells}, Treatment: {Treatment_conditions}, Repeat: {repeat} - Expected 4 Flow_speeds, found {group['Flow_speed'].nunique()}")
        break  # Stop the entire process if any inconsistency is found

if consistent_flow_speeds:
    print("Each combination of 'Cells', 'Treatment', 'Repeat' has exactly 4 different 'Flow_speed' values.")
else:
    print("There are inconsistencies in 'Flow_speed' values. Please check the output for details.")


unique_cells = unique_files_df['Cells'].unique()
unique_flow_speeds = unique_files_df['Flow_speed'].unique()
unique_Treatment = unique_files_df['Treatment'].unique()
unique_conditions = unique_files_df['Condition'].unique()

print("Unique Cells:", unique_cells)
print("Unique Flow Speeds:", unique_flow_speeds)
print("Unique Treatment:", unique_Treatment)
print("Unique Conditions:", unique_conditions)


## **1.4. Filter tracks shorter than 50 spots**


In [ ]:
# @title ##Filter tracks shorter than 50 spots


merged_tracks_df = merged_tracks_df[merged_tracks_df['NUMBER_SPOTS'] >= 50]
merged_spots_df = merged_spots_df[merged_spots_df['Unique_ID'].isin(merged_tracks_df['Unique_ID'])]


## **1.5. Visualise your tracks**
---

In [ ]:
# @title ##Run the cell and choose the file you want to inspect

import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

if not os.path.exists(Results_Folder+"/Tracks"):
    os.makedirs(Results_Folder+"/Tracks")  # Create Results_Folder if it doesn't exist

# Extract unique filenames from the dataframe
filenames = merged_spots_df['File_name'].unique()

# Create a Dropdown widget with the filenames
filename_dropdown = widgets.Dropdown(
    options=filenames,
    value=filenames[0] if len(filenames) > 0 else None,  # Default selected value
    description='File Name:',
)

def plot_coordinates(filename):
    if filename:
        # Filter the DataFrame based on the selected filename
        filtered_df = merged_spots_df[merged_spots_df['File_name'] == filename]

        plt.figure(figsize=(10, 8))
        for unique_id in filtered_df['Unique_ID'].unique():
            unique_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')
            plt.plot(unique_df['POSITION_X'], unique_df['POSITION_Y'], marker='o', linestyle='-', markersize=2)

        plt.xlabel('POSITION_X')
        plt.ylabel('POSITION_Y')
        plt.title(f'Coordinates for {filename}')
        plt.savefig(f"{Results_Folder}/Tracks/Tracks_{filename}.pdf")
        plt.show()
    else:
        print("No valid filename selected")

# Link the Dropdown widget to the plotting function
interact(plot_coordinates, filename=filename_dropdown)


--------------------------------------------------------
# **Part 2: Assess spatial clustering using Ripley's L function**
--------------------------------------------------------

<font size = 4>**Spatial Clustering Analyses:** For a comprehensive guide on performing spatial clustering analysis with CellTracksColab, visit the project's [Spatial Clustering analyses wiki page](https://github.com/CellMigrationLab/CellTracksColab/wiki/Spatial-Clustering-analyses).


## **2.1. Filter tracks with Track_MIN_speed**

<font size = 4>This section enables to filter the dataset so that we only keep arresting tracks.





In [ ]:
# @title ##Filter tracks using Min Speed


merged_tracks_df = merged_tracks_df[merged_tracks_df['Min Speed'] <= 5]
merged_spots_df = merged_spots_df[merged_spots_df['Unique_ID'].isin(merged_tracks_df['Unique_ID'])]

## **2.2. Visualise where cells slow down in each tracks**


In [ ]:
# @title ##Run the cell and choose the file you want to inspect to visualise track and choosen point


import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import os

if not os.path.exists(Results_Folder+"/Tracks"):
    os.makedirs(Results_Folder+"/Tracks")  # Create Results_Folder if it doesn't exist

# Extract unique filenames from the dataframe
filenames = merged_spots_df['File_name'].unique()

# Create a Dropdown widget with the filenames
filename_dropdown = widgets.Dropdown(
    options=filenames,
    value=filenames[0] if len(filenames) > 0 else None,  # Default selected value
    description='File Name:',
)

# User-defined speed threshold
speed_threshold = 3  # Replace with the desired threshold value

def find_point_below_threshold(track):
    below_threshold = track[track['Speed'] < speed_threshold]
    return below_threshold.iloc[0] if not below_threshold.empty else None

def plot_coordinates(filename):
    if filename:
        # Filter the DataFrame based on the selected filename
        filtered_df = merged_spots_df[merged_spots_df['File_name'] == filename]

        plt.figure(figsize=(10, 8))
        for unique_id in filtered_df['Unique_ID'].unique():
            unique_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')
            plt.plot(unique_df['POSITION_X'], unique_df['POSITION_Y'], marker='o', linestyle='-', markersize=2)

            # Find and mark the slowdown point
            slowdown_point = find_point_below_threshold(unique_df)
            if slowdown_point is not None:
                plt.scatter(slowdown_point['POSITION_X'], slowdown_point['POSITION_Y'], color='red', s=50)
            #else:
                #print(f"No slowdown point found for track {unique_id}")
        plt.xlabel('POSITION_X')
        plt.ylabel('POSITION_Y')
        plt.title(f'Coordinates for {filename}')
        plt.savefig(f"{Results_Folder}/Tracks/Tracks_{filename}.pdf")
        plt.show()
    else:
        print("No valid filename selected")


# Link the Dropdown widget to the plotting function
interact(plot_coordinates, filename=filename_dropdown)


## **2.3 Identify the coordinates where cells slow down**

<font size = 4>Here we identify the where circulating cells slow down on the endothelial monolayer to identify possible hotspots.


In [ ]:
# @title ##Run the cell to identify the coordinates to use for the clustering analysis


import matplotlib.pyplot as plt
import os

if not os.path.exists(Results_Folder + "/Slow_down_coordinates"):
    os.makedirs(Results_Folder + "/Slow_down_coordinates")  # Create Results_Folder if it doesn't exist

# Extract unique filenames from the dataframe
filenames = merged_spots_df['File_name'].unique()

# User-defined speed threshold
speed_threshold = 5  # Replace with the desired threshold value

def find_point_below_threshold(track):
    below_threshold = track[track['Speed'] < speed_threshold]
    return below_threshold.iloc[0] if not below_threshold.empty else None

def plot_slowdown_points(filename):
    if filename:
        # Filter the DataFrame based on the filename
        filtered_df = merged_spots_df[merged_spots_df['File_name'] == filename]

        plt.figure(figsize=(10, 8))
        for unique_id in filtered_df['Unique_ID'].unique():
            unique_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')

            # Find and mark the slowdown point
            slowdown_point = find_point_below_threshold(unique_df)
            if slowdown_point is not None:
                plt.scatter(slowdown_point['POSITION_X'], slowdown_point['POSITION_Y'], color='red', s=50, label=f'Track {unique_id}')

        plt.xlabel('POSITION_X')
        plt.ylabel('POSITION_Y')
        plt.title(f'Slowdown Points for {filename}')
        plt.savefig(f"{Results_Folder}/Slow_down_coordinates/Slowdown_Points_{filename}.pdf")
        plt.close()
    else:
        print("No valid filename selected")

# Loop through each file and generate the plot for slowdown points
for filename in filenames:
    plot_slowdown_points(filename)



## **2.4 Compute the Ripley's L function for each FOV**


In [ ]:
# @title ##Compute Ripley's L function for each FOV

# User-defined speed threshold
speed_threshold = 5

# Check and create necessary directories
if not os.path.exists(f"{Results_Folder}/Track_Clustering"):
    os.makedirs(f"{Results_Folder}/Track_Clustering")

import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt

# Define Ripley's K function
def ripley_k(points, r, area):
    n = len(points)
    d_matrix = distance_matrix(points, points)
    sum_indicator = np.sum(d_matrix < r) - n  # Subtract n to exclude self-pairs

    K_r = (area / (n ** 2)) * sum_indicator

    # Check if K_r is negative and print relevant information
    if K_r < 0:
        print("Negative K_r encountered!")
        print("Distance matrix:", d_matrix)
        print("Sum indicator:", sum_indicator)
        print("Area:", area, "Number of points:", n, "Distance threshold r:", r)

    return K_r


# Define Ripley's L function

def ripley_l(points, r, area):
    K_r = ripley_k(points, r, area)
    # Check if K_r has negative values
    if np.any(K_r < 0):
        print("Warning: Negative value encountered in K_r")

    L_r = np.sqrt(K_r / np.pi) - r
    return L_r

def find_point_below_threshold(track):
  below_threshold = track[track['Speed'] < speed_threshold]
  if not below_threshold.empty:
    return below_threshold.iloc[0][['POSITION_X', 'POSITION_Y']]
  return pd.Series([np.nan, np.nan], index=['POSITION_X', 'POSITION_Y'])

# Define area based on your dataset's extent
area = (merged_spots_df['POSITION_X'].max() - merged_spots_df['POSITION_X'].min()) * \
       (merged_spots_df['POSITION_Y'].max() - merged_spots_df['POSITION_Y'].min())

# Define r values
r_values = np.linspace(1, 250, 250)  # Adjust as needed

# Compute Ripley's L function for each FOV
l_values_per_fov_slow = {}
for file_name, group in tqdm(merged_spots_df.groupby('File_name'), desc="Processing FOVs"):
    # Sort each track by POSITION_T
    group = group.sort_values(by=['TRACK_ID', 'POSITION_T'])

    representative_points = group.groupby('TRACK_ID').apply(find_point_below_threshold).dropna()
    if not representative_points.empty:
        l_values = [ripley_l(representative_points.values, r, area) for r in tqdm(r_values, desc=f"Calculating L for {file_name}")]
        l_values_per_fov_slow[file_name] = l_values


## **2.5 Compute Monte Carlo simulations for each FOV**

In [ ]:
from tqdm.notebook import tqdm

# @title ##Compute Monte Carlo simulations for each FOV


# Simulate random points for Monte Carlo simulations
def simulate_random_points(num_points, x_range, y_range):
    x_coords = np.random.uniform(x_range[0], x_range[1], num_points)
    y_coords = np.random.uniform(y_range[0], y_range[1], num_points)
    return np.column_stack((x_coords, y_coords))

# Initialize simulated_l_values as an empty dictionary
simulated_l_values_dict_slow = {}

# Perform Monte Carlo simulations for significance testing
confidence_envelopes_slow = {}
for file_name, group in tqdm(merged_spots_df.groupby('File_name'), desc='Processing FOVs'):

    group = group.sort_values(by=['TRACK_ID', 'POSITION_T'])
    representative_points = group.groupby('TRACK_ID').apply(find_point_below_threshold).dropna()

    simulations = [simulate_random_points(len(representative_points),
                                          (merged_spots_df['POSITION_X'].min(), merged_spots_df['POSITION_X'].max()),
                                          (merged_spots_df['POSITION_Y'].min(), merged_spots_df['POSITION_Y'].max()))
                   for _ in tqdm(range(10), desc=f'Simulating for {file_name}', leave=False)]

    simulated_l_values = [[ripley_l(points, r, area) for r in r_values] for points in simulations]
    simulated_l_values_dict_slow[file_name] = simulated_l_values  # Store the simulated values in the dictionary

    lower_bound = np.percentile(simulated_l_values, 2.5, axis=0)
    upper_bound = np.percentile(simulated_l_values, 97.5, axis=0)
    confidence_envelopes_slow[file_name] = (lower_bound, upper_bound)



## **2.6 Plot the results for each FOV**

In [ ]:
# @title ##Plots for each FOV - Slow down

import os
import matplotlib.pyplot as plt

# Visualization of Ripley's L function with confidence envelopes
for file_name, l_values in l_values_per_fov_slow.items():
    # Retrieve the confidence envelope for the current file
    lower_bound, upper_bound = confidence_envelopes_slow.get(file_name, (None, None))

    # Only proceed if the confidence envelope exists
    if lower_bound is not None and upper_bound is not None:
        plt.figure(figsize=(10, 6))
        plt.plot(r_values, l_values, label=f'L(r) for {file_name}')
        plt.fill_between(r_values, lower_bound, upper_bound, color='gray', alpha=0.5)
        plt.xlabel('Radius (r)')
        plt.ylabel("Ripley's L Function")
        plt.title(f"Ripley's L Function - {file_name}")
        plt.legend()
        plt.grid(True)

        # Save the plot as a PDF in the specified folder
        pdf_path = os.path.join(f"{Results_Folder}/Track_Clustering/{file_name}.pdf")
        plt.savefig(pdf_path,bbox_inches='tight')
        plt.show()
        plt.close()  # Close the plot to free memory
    else:
        print(f"No confidence envelope data available for {file_name}")


## **2.7 Define a specific radius and save as dataframe**

This is performed to compare FOV and conditions

In [ ]:
# @title ##Define a specific radius and save as dataframe - Slow down


# Define the specific radius for comparison
specific_radius = 50  # Replace with your chosen radius

# Extract L values at the specific radius
specific_radius_index = np.argmin(np.abs(r_values - specific_radius))  # Find the index of the closest radius value
l_values_at_specific_radius_slow = {fov: l_values[specific_radius_index] for fov, l_values in l_values_per_fov_slow.items()}

# Plotting
plt.figure(figsize=(12, 6))
plt.bar(l_values_at_specific_radius_slow.keys(), l_values_at_specific_radius_slow.values())
plt.xlabel('Field of View')
plt.ylabel(f"Ripley's L at radius {specific_radius}")
plt.title(f"Comparison of Ripley's L Function at Radius {specific_radius} Across Different FOVs")
plt.xticks(rotation=45)
# Save the plot as a PDF in the specified folder
pdf_path = os.path.join(f"{Results_Folder}/Track_Clustering/l_values_at_specific_radius_slow.pdf")
plt.savefig(pdf_path, bbox_inches='tight')

plt.show()


# Create DataFrame with confidence envelopes, median, and L values at the specific radius
rows = []
for fov, (lower_bound, upper_bound) in confidence_envelopes_slow.items():
    l_value = l_values_per_fov_slow[fov][specific_radius_index]
    lower = lower_bound[specific_radius_index]
    upper = upper_bound[specific_radius_index]

    # Retrieve simulated L values for the FOV
    simulated_l_values_for_fov_slow = simulated_l_values_dict_slow.get(fov, [])

    # Calculate median if simulated L values are available for the FOV
    if simulated_l_values_for_fov_slow:
        median_vals = [l_vals[specific_radius_index] for l_vals in simulated_l_values_for_fov_slow]
        median = np.median(median_vals) if median_vals else np.nan
    else:
        median = np.nan

    rows.append([fov, l_value, lower, upper, median])

confidence_df = pd.DataFrame(rows, columns=['File_name', 'Ripley_L_at_Specific_Radius_slow', 'Lower_Bound_slow', 'Upper_Bound_slow', 'Median_slow'])

# Merge with additional information
additional_info_df = merged_tracks_df[['File_name', 'Cells', 'Flow_speed', 'Treatment', 'Condition', 'experiment_nb', 'Repeat']].drop_duplicates('File_name')
merged_df = pd.merge(confidence_df, additional_info_df, left_on='File_name', right_on='File_name')

# Save the merged DataFrame to a CSV file
merged_df.to_csv(f"{Results_Folder}/Track_Clustering/ripleys_l_values.csv", index=False)


## **2.8 Ripley's L Values Across conditions and cells**


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# @title ##Comparison of Ripley\'s L Values Across Conditions

# Convert 'Condition' to string if it's not already
merged_df['Condition'] = merged_df['Condition'].astype(str)

# Create the box plot
plt.figure(figsize=(12, 8))
sns.boxplot(data=merged_df, x='Condition', y='Ripley_L_at_Specific_Radius_slow')

# Overlay the Monte Carlo simulation results
for condition in merged_df['Condition'].unique():
    condition_data = merged_df[merged_df['Condition'] == condition]

    # Plot median values
    medians = condition_data['Median_slow']
    plt.scatter([condition] * len(medians), medians, color='red', alpha=0.5)  # Median

    # Handle NaN values and calculate mean and error only for non-NaN values
    valid_data = condition_data.dropna(subset=['Median_slow', 'Lower_Bound_slow', 'Upper_Bound_slow'])
    if not valid_data.empty:
        median_mean = valid_data['Median_slow'].mean()
        lower_mean = valid_data['Lower_Bound_slow'].mean()
        upper_mean = valid_data['Upper_Bound_slow'].mean()
        yerr = [[median_mean - lower_mean], [upper_mean - median_mean]]

        # Check if yerr contains valid data before plotting
        if not any(np.isnan(yerr)):
            plt.errorbar(condition, median_mean, yerr=yerr, fmt='o', color='black', alpha=0.5)  # Confidence interval

# Add labels and title
plt.xlabel('Condition')
plt.ylabel('Ripley\'s L at Specific Radius')
plt.title('Comparison of Ripley\'s L Values Across Conditions with Monte Carlo Simulation Results')
plt.xticks(rotation=45)
plt.grid(True)

# Save the figure before showing it
pdf_path = os.path.join(f"{Results_Folder}/Track_Clustering/l_values_Conditions_slow.pdf")
plt.savefig(pdf_path, bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# @title ##Comparison of Ripley\'s L Values Across Cells

# Convert 'Condition' to string if it's not already
merged_df['Cells'] = merged_df['Cells'].astype(str)

# Create the box plot
plt.figure(figsize=(12, 8))
sns.boxplot(data=merged_df, x='Cells', y='Ripley_L_at_Specific_Radius_slow')

# Overlay the Monte Carlo simulation results
for condition in merged_df['Cells'].unique():
    condition_data = merged_df[merged_df['Cells'] == condition]

    # Plot median values
    medians = condition_data['Median_slow']
    plt.scatter([condition] * len(medians), medians, color='red', alpha=0.5)  # Median

    # Handle NaN values and calculate mean and error only for non-NaN values
    valid_data = condition_data.dropna(subset=['Median_slow', 'Lower_Bound_slow', 'Upper_Bound_slow'])
    if not valid_data.empty:
        median_mean = valid_data['Median_slow'].mean()
        lower_mean = valid_data['Lower_Bound_slow'].mean()
        upper_mean = valid_data['Upper_Bound_slow'].mean()
        yerr = [[median_mean - lower_mean], [upper_mean - median_mean]]

        # Check if yerr contains valid data before plotting
        if not any(np.isnan(yerr)):
            plt.errorbar(condition, median_mean, yerr=yerr, fmt='o', color='black', alpha=0.5)  # Confidence interval

# Add labels and title
plt.xlabel('Condition')
plt.ylabel('Ripley\'s L at Specific Radius')
plt.title('Comparison of Ripley\'s L Values Across Conditions with Monte Carlo Simulation Results')
plt.xticks(rotation=45)
plt.grid(True)

# Save the figure before showing it
pdf_path = os.path.join(f"{Results_Folder}/Track_Clustering/l_values_Cells_slow.pdf")
plt.savefig(pdf_path, bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
# @title ##Comparison of Ripley\'s L Values Across Cells and Treatment


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Convert 'Cells' and 'Treatment' to string if they are not already
merged_df['Cells'] = merged_df['Cells'].astype(str)
merged_df['Treatment'] = merged_df['Treatment'].astype(str)

# Create a combined factor for Cells and Silencing
merged_df['Cells_Treatment'] = merged_df['Cells'] + "_" + merged_df['Treatment']

# Create the box plot
plt.figure(figsize=(14, 8))
sns.boxplot(data=merged_df, x='Cells_Treatment', y='Ripley_L_at_Specific_Radius_slow')

# Overlay the Monte Carlo simulation results
for condition in merged_df['Cells_Treatment'].unique():
    condition_data = merged_df[merged_df['Cells_Treatment'] == condition]

    # Plot median values
    medians = condition_data['Median_slow']
    plt.scatter([condition] * len(medians), medians, color='red', alpha=0.5)  # Median

    # Handle NaN values and calculate mean and error only for non-NaN values
    valid_data = condition_data.dropna(subset=['Median_slow', 'Lower_Bound_slow', 'Upper_Bound_slow'])
    if not valid_data.empty:
        median_mean = valid_data['Median_slow'].mean()
        lower_mean = valid_data['Lower_Bound_slow'].mean()
        upper_mean = valid_data['Upper_Bound_slow'].mean()
        yerr = [[median_mean - lower_mean], [upper_mean - median_mean]]

        # Check if yerr contains valid data before plotting
        if not any(np.isnan(yerr)):
            plt.errorbar(condition, median_mean, yerr=yerr, fmt='o', color='black', alpha=0.5)  # Confidence interval

# Add labels and title
plt.xlabel('Cells and Treatment')
plt.ylabel('Ripley\'s L at Specific Radius')
plt.title('Comparison of Ripley\'s L Values Across Cells and Silencing Conditions with Monte Carlo Simulation Results')
plt.xticks(rotation=45)
plt.grid(True)

# Save the figure before showing it
pdf_path = os.path.join(f"{Results_Folder}/Track_Clustering/l_values_Cells_Treatment_slow.pdf")
plt.savefig(pdf_path, bbox_inches='tight')

# Show the plot
plt.show()


--------------------------------------------------------
# **Part 3: Cluster vs single cell arrest (space–time co-arrest)**
--------------------------------------------------------

<font size = 4>

## **3.1. Filter tracks with Track_MIN_speed**

<font size = 4>This section enables to filter the dataset so that we only keep arresting tracks.

In [ ]:
# @title ##Filter tracks using Min Speed


merged_tracks_df = merged_tracks_df[merged_tracks_df['Min Speed'] <= 5]
merged_spots_df = merged_spots_df[merged_spots_df['Unique_ID'].isin(merged_tracks_df['Unique_ID'])]

## **3.2 Identify the coordinates where cells slow down**

<font size = 4>Here we identify the where circulating cells slow down on the endothelial monolayer to identify possible hotspots.


In [ ]:
# @title ## Slowdown maps colored by time

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

# --- Output folder (different from Slow_down_coordinates) ---
OUT_DIR = os.path.join(Results_Folder, "Slowdown_TimeColored")
os.makedirs(OUT_DIR, exist_ok=True)

# --- User-defined threshold (keep consistent with your filter above) ---
speed_threshold = 5  # same units as merged_spots_df['Speed']

def _sanitize_filename(s):
    """Make a safe filename fragment from the FOV name."""
    return str(s).replace(os.sep, "_").replace("/", "_").replace("\\", "_")

def get_first_slowdown_points_for_fov(filename, df_spots, threshold):
    """
    For a given FOV (File_name), return a dataframe with the first point
    where each track's Speed drops below `threshold`.
    Columns: Unique_ID, File_name, t0 (POSITION_T), x0 (POSITION_X), y0 (POSITION_Y)
    """
    fov_df = df_spots[df_spots['File_name'] == filename]
    events = []
    # groupby Unique_ID to find first slowdown per track
    for uid, track in fov_df.groupby('Unique_ID', sort=False):
        track = track.sort_values('POSITION_T')
        below = track[track['Speed'] <= threshold]
        if not below.empty:
            row = below.iloc[0]
            events.append({
                'Unique_ID': uid,
                'File_name': filename,
                't0': float(row['POSITION_T']),
                'x0': float(row['POSITION_X']),
                'y0': float(row['POSITION_Y'])
            })
    return pd.DataFrame(events)

# --- Build slowdown events for all FOVs & save time-colored plots ---
all_events = []
filenames = merged_spots_df['File_name'].dropna().unique()

# If you prefer a consistent color scale across FOVs, set this to True.
USE_GLOBAL_TIME_SCALE = False

# First pass (optional) to compute global t-range for a consistent colorbar
tmin_global, tmax_global = None, None
if USE_GLOBAL_TIME_SCALE:
    tmp_list = []
    for fn in filenames:
        ev = get_first_slowdown_points_for_fov(fn, merged_spots_df, speed_threshold)
        if not ev.empty:
            tmp_list.append(ev['t0'])
    if tmp_list:
        tmin_global = float(pd.concat(tmp_list).min())
        tmax_global = float(pd.concat(tmp_list).max())

for fn in filenames:
    ev = get_first_slowdown_points_for_fov(fn, merged_spots_df, speed_threshold)
    all_events.append(ev)

    # Save per-FOV CSV (handy for later clustering step)
    if not ev.empty:
        ev_out = os.path.join(OUT_DIR, f"slowdown_events_{_sanitize_filename(fn)}.csv")
        ev.to_csv(ev_out, index=False)

        # Plot: dots colored by slowdown time
        fig, ax = plt.subplots(figsize=(10, 8))
        if USE_GLOBAL_TIME_SCALE and (tmin_global is not None):
            norm = mpl.colors.Normalize(vmin=tmin_global, vmax=tmax_global)
        else:
            norm = None

        sc = ax.scatter(ev['x0'], ev['y0'],
                        c=ev['t0'],
                        cmap='viridis',
                        norm=norm,
                        s=40,
                        edgecolors='none')
        cbar = plt.colorbar(sc, ax=ax)
        cbar.set_label("First slowdown time (POSITION_T)")

        ax.set_xlabel("POSITION_X")
        ax.set_ylabel("POSITION_Y")
        ax.set_title(f"Slowdown points colored by time (< {speed_threshold})\n{fn}")

        fig.tight_layout()
        out_pdf = os.path.join(OUT_DIR, f"Slowdown_TimeColored_{_sanitize_filename(fn)}.pdf")
        fig.savefig(out_pdf)
        plt.close(fig)

# Save an all-FOV events table (optional convenience)
if len(all_events):
    slowdown_events_df = pd.concat(all_events, ignore_index=True)
    slowdown_events_df.to_csv(os.path.join(OUT_DIR, "slowdown_events_all.csv"), index=False)

print(f"Saved time-colored slowdown maps and CSVs to: {OUT_DIR}")


## **3.3 Identification of co-arrest**




In [ ]:
# @title ## Co-arrest fraction (radius = k × cell diameter per FOV)

import os
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

# ---- PARAMETERS ----
DELTA_T_UNITS = 0.08   # time window; SAME UNITS as POSITION_T
# One diameter (µm) per cell type in merged_tracks_df['Cells']  <-- FILL THESE
DIAMETER_MAP_UM = {
     "AsPc1": 17,
     "MiaPaca-2": 20,
    "Panc10": 20,
     "Monocyte": 15,
     "Neutrophil": 12
}
K_RADIUS = 1.0  # neighborhood size in cell diameters (try 0.75 / 1.0 / 1.5)

# ---- INPUT EVENTS ----
EVENTS_DIR = os.path.join(Results_Folder, "Slowdown_TimeColored")
OUT_DIR_CO = os.path.join(Results_Folder, "CoArrest_TimeWindow")
os.makedirs(OUT_DIR_CO, exist_ok=True)

def load_slowdown_events(events_dir: str) -> pd.DataFrame:
    all_path = os.path.join(events_dir, "slowdown_events_all.csv")
    if os.path.exists(all_path):
        return pd.read_csv(all_path)
    frames = []
    for fn in os.listdir(events_dir):
        if fn.startswith("slowdown_events_") and fn.endswith(".csv"):
            frames.append(pd.read_csv(os.path.join(events_dir, fn)))
    if frames:
        return pd.concat(frames, ignore_index=True)
    raise RuntimeError("No slowdown events found. Run the slowdown export first.")

def co_arrest_flags_for_fov(ev_fov: pd.DataFrame, r: float, dt: float) -> np.ndarray:
    n = len(ev_fov)
    if n <= 1:
        return np.zeros(n, dtype=bool)
    X = ev_fov[['x0','y0']].to_numpy(float)
    T = ev_fov['t0'].to_numpy(float)
    D = cdist(X, X)
    dT = np.abs(T[:, None] - T[None, :])
    A = (D <= r) & (dT <= dt)
    np.fill_diagonal(A, False)
    return A.any(axis=1)

# ---- Build per-FOV radius from cell diameter ----
meta_by_fov_cells = (
    merged_tracks_df
    .groupby('File_name', as_index=False)[['Cells']]
    .agg(lambda s: s.iloc[0])
)
meta_by_fov_cells['Cells'] = meta_by_fov_cells['Cells'].astype(str)

unknown = sorted(set(meta_by_fov_cells['Cells'].unique()) - set(DIAMETER_MAP_UM.keys()))
if unknown:
    raise ValueError(f"Please add diameters for these Cells labels in DIAMETER_MAP_UM: {unknown}")

meta_by_fov_cells['diam_um'] = meta_by_fov_cells['Cells'].map(DIAMETER_MAP_UM).astype(float)
meta_by_fov_cells['r_fov_um'] = K_RADIUS * meta_by_fov_cells['diam_um']
r_map = dict(zip(meta_by_fov_cells['File_name'], meta_by_fov_cells['r_fov_um']))

# ---- Load events, compute flags, summarize ----
events_df = load_slowdown_events(EVENTS_DIR).copy()
required = {'File_name','t0','x0','y0'}
missing = required - set(events_df.columns)
if missing:
    raise ValueError(f"Missing columns in slowdown events: {missing}")

per_fov_rows, flag_series = [], []
for fov, g in events_df.groupby('File_name'):
    r_fov = float(r_map[fov])
    flags = co_arrest_flags_for_fov(g, r_fov, DELTA_T_UNITS)
    flag_series.append(pd.Series(flags, index=g.index))

    total = len(g)
    together = int(flags.sum())
    alone = total - together
    frac_together = (together / total) if total else np.nan

    per_fov_rows.append({
        "File_name": fov,
        "Cells": str(meta_by_fov_cells.loc[meta_by_fov_cells['File_name']==fov, 'Cells'].iloc[0]),
        "diam_um": float(meta_by_fov_cells.loc[meta_by_fov_cells['File_name']==fov, 'diam_um'].iloc[0]),
        "r_fov_um": r_fov,
        "k_radius": K_RADIUS,
        "delta_t_units": DELTA_T_UNITS,
        "total_events": total,
        "together_events": together,
        "alone_events": alone,
        "frac_together": frac_together,
        "frac_alone": 1.0 - frac_together if total else np.nan,
        "pct_together": 100.0 * frac_together if total else np.nan,
        "pct_alone": 100.0 * (1.0 - frac_together) if total else np.nan,
    })

events_df["r_fov_um"] = events_df["File_name"].map(r_map)
events_df["k_radius"] = K_RADIUS
events_df["delta_t_units"] = DELTA_T_UNITS

TAG = f"rk{K_RADIUS}_dt{DELTA_T_UNITS}"
summary_df = pd.DataFrame(per_fov_rows).sort_values("File_name")

summary_path = os.path.join(OUT_DIR_CO, f"co_arrest_summary_by_FOV_{TAG}.csv")
events_path  = os.path.join(OUT_DIR_CO, f"slowdown_events_with_coarrestflags_{TAG}.csv")
summary_df.to_csv(summary_path, index=False)
events_df.to_csv(events_path, index=False)

# ---- Overall headline (weighted by events) ----
overall_total = int(summary_df["total_events"].sum())
overall_together = int(summary_df["together_events"].sum())
overall_alone = int(summary_df["alone_events"].sum())
overall_frac_together = (overall_together / overall_total) if overall_total else np.nan
overall_frac_alone    = (overall_alone / overall_total) if overall_total else np.nan

overall_path = os.path.join(OUT_DIR_CO, f"overall_co_arrest_totals_{TAG}.csv")
pd.DataFrame([{
    "overall_total_events": overall_total,
    "overall_together_events": overall_together,
    "overall_alone_events": overall_alone,
    "overall_frac_together": overall_frac_together,
    "overall_frac_alone": overall_frac_alone,
    "overall_pct_together": 100.0 * overall_frac_together if overall_total else np.nan,
    "overall_pct_alone": 100.0 * overall_frac_alone if overall_total else np.nan,
    "k_radius": K_RADIUS,
    "delta_t_units": DELTA_T_UNITS,
}]).to_csv(overall_path, index=False)

# ---- Quick overall bar chart (optional) ----
fig, ax = plt.subplots(figsize=(4.5,4))
ax.bar(["Alone","Together"], [overall_frac_alone, overall_frac_together])
ax.set_ylim(0, 1)
ax.set_ylabel("Fraction of arrests")
ax.set_title(f"Co-arrest fraction (r = k×diameter; k={K_RADIUS}, ΔT={DELTA_T_UNITS})")
fig.tight_layout()
bar_path = os.path.join(OUT_DIR_CO, f"overall_co_arrest_fraction_{TAG}.png")
fig.savefig(bar_path, dpi=200)
plt.close(fig)

print("Saved:", summary_path)
print("Saved:", events_path)
print("Saved:", overall_path)
print("Saved:", bar_path)


In [ ]:
# @title ## QC: Sensitivity across k (radius in cell diameters) and ΔT
import os
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

# --- Locate a per-event table with t0/x0/y0 (prefer the one from Cell 3) ---
try:
    base_events = pd.read_csv(events_path)[['File_name','Unique_ID','t0','x0','y0']]
    print(f"[QC] Loaded events from events_path: {events_path}")
except Exception:
    # Fallback 1: most recent flagged events in CoArrest_TimeWindow
    co_dir = os.path.join(Results_Folder, "CoArrest_TimeWindow")
    cand = [fn for fn in os.listdir(co_dir)
            if fn.startswith("slowdown_events_with_coarrestflags_") and fn.endswith(".csv")]
    if cand:
        cand.sort()
        fp = os.path.join(co_dir, cand[-1])
        base_events = pd.read_csv(fp)[['File_name','Unique_ID','t0','x0','y0']]
        print(f"[QC] Loaded events from fallback: {fp}")
    else:
        # Fallback 2: raw slowdown events
        fp = os.path.join(Results_Folder, "Slowdown_TimeColored", "slowdown_events_all.csv")
        base_events = pd.read_csv(fp)[['File_name','Unique_ID','t0','x0','y0']]
        print(f"[QC] Loaded events from raw slowdown file: {fp}")

# --- Build a per-FOV diameter map (µm) ---
# Prefer the table made earlier; otherwise rebuild from merged_tracks_df + DIAMETER_MAP_UM
if 'meta_by_fov_cells' in globals() and 'diam_um' in meta_by_fov_cells.columns:
    diam_map = dict(zip(meta_by_fov_cells['File_name'], meta_by_fov_cells['diam_um']))
else:
    if 'DIAMETER_MAP_UM' not in globals():
        raise RuntimeError("DIAMETER_MAP_UM not defined. Define it as {Cells_label: diameter_um}.")
    meta_by_fov_cells = (
        merged_tracks_df
        .groupby('File_name', as_index=False)[['Cells']]
        .agg(lambda s: s.iloc[0])
    )
    meta_by_fov_cells['Cells'] = meta_by_fov_cells['Cells'].astype(str)
    unknown = sorted(set(meta_by_fov_cells['Cells'].unique()) - set(DIAMETER_MAP_UM.keys()))
    if unknown:
        raise ValueError(f"Please add diameters for these Cells labels in DIAMETER_MAP_UM: {unknown}")
    meta_by_fov_cells['diam_um'] = meta_by_fov_cells['Cells'].map(DIAMETER_MAP_UM).astype(float)
    diam_map = dict(zip(meta_by_fov_cells['File_name'], meta_by_fov_cells['diam_um']))

# --- Helper: fraction 'together' for a single FOV given k & ΔT ---
def frac_together_k(ev_fov: pd.DataFrame, k: float, dt: float, diam_um: float) -> float:
    n = len(ev_fov)
    if n <= 1 or not np.isfinite(diam_um):
        return np.nan
    r = k * float(diam_um)  # radius in µm (positions already calibrated)
    X = ev_fov[['x0','y0']].to_numpy(float)
    T = ev_fov['t0'].to_numpy(float)
    D = cdist(X, X)
    dT = np.abs(T[:, None] - T[None, :])
    A = (D <= r) & (dT <= dt)
    np.fill_diagonal(A, False)
    return float(A.any(axis=1).mean())

# --- Grid centered on your current K_RADIUS from Cell 3 ---
if 'K_RADIUS' not in globals():
    K_RADIUS = 1.0  # fallback
grid_k  = [0.75 * K_RADIUS, K_RADIUS, 1.5 * K_RADIUS]
grid_dt = [0.5 * DELTA_T_UNITS, DELTA_T_UNITS, 2.0 * DELTA_T_UNITS]

rows = []
for k in grid_k:
    for dt in grid_dt:
        vals = []
        for fov, g in base_events.groupby('File_name'):
            diam = diam_map.get(fov, np.nan)
            vals.append(frac_together_k(g, k, dt, diam))
        rows.append({'k': k, 'dt': dt, 'mean_frac_together': np.nanmean(vals)})

sens_df = pd.DataFrame(rows)
pivot = sens_df.pivot(index='k', columns='dt', values='mean_frac_together').round(3)
print(pivot)

# --- Save results (CSV + small heatmap PDF) ---
qc_csv = os.path.join(OUT_DIR_CO, "qc_sensitivity_k_dt.csv")
sens_df.to_csv(qc_csv, index=False)

fig, ax = plt.subplots(figsize=(5, 4))
im = ax.imshow(pivot.values, aspect='auto', origin='lower')
ax.set_xticks(range(len(pivot.columns)))
ax.set_yticks(range(len(pivot.index)))
ax.set_xticklabels([f"{v:.3g}" for v in pivot.columns], rotation=45, ha='right')
ax.set_yticklabels([f"{v:.3g}" for v in pivot.index])
ax.set_xlabel("ΔT (same units as POSITION_T)")
ax.set_ylabel("k (× cell diameter)")
ax.set_title("Sensitivity of mean co-arrest fraction")
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("Mean frac_together")
fig.tight_layout()
qc_pdf = os.path.join(OUT_DIR_CO, "qc_sensitivity_k_dt_heatmap.pdf")
fig.savefig(qc_pdf)
plt.close(fig)

print("Saved sensitivity table to:", qc_csv)
print("Saved sensitivity heatmap to:", qc_pdf)


In [ ]:
# @title ## Plot Co-arrest fraction by Cells / Condition / Treatment / Cells_Treatment (FOV-level)

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ---- INPUTS ----
META_COLS = ['Cells', 'Treatment', 'Condition', 'Flow_speed', 'experiment_nb', 'Repeat']

COARREST_DIR = os.path.join(Results_Folder, "CoArrest_TimeWindow")
cand = [fn for fn in os.listdir(COARREST_DIR) if fn.startswith("co_arrest_summary_by_FOV_") and fn.endswith(".csv")]
if not cand:
    raise RuntimeError("No co_arrest_summary_by_FOV_*.csv found. Run the co-arrest cell first.")
cand.sort()
summary_path = os.path.join(COARREST_DIR, cand[-1])
summary_df = pd.read_csv(summary_path)

# Try to fetch k and dt from the summary (for plot titles); fallback to globals if present
k_in = summary_df['k_radius'].iloc[0] if 'k_radius' in summary_df.columns else (K_RADIUS if 'K_RADIUS' in globals() else None)
dt_in = summary_df['delta_t_units'].iloc[0] if 'delta_t_units' in summary_df.columns else (DELTA_T_UNITS if 'DELTA_T_UNITS' in globals() else None)

def _title_suffix():
    bits = []
    if k_in is not None: bits.append(f"k={k_in:g}")
    if dt_in is not None: bits.append(f"ΔT={dt_in}")
    return f" ({', '.join(bits)})" if bits else ""

# ---- OUTPUT FOLDER ----
OUT_DIR = os.path.join(Results_Folder, "CoArrest_TimeWindow_ByCondition")
os.makedirs(OUT_DIR, exist_ok=True)

# ---- Merge metadata (per-FOV) ----
meta_by_fov = (
    merged_tracks_df
    .groupby('File_name', as_index=False)[META_COLS]
    .agg(lambda s: s.iloc[0])
)

coarrest_with_meta = summary_df.merge(meta_by_fov, on='File_name', how='left')

# --- Helper to coalesce duplicate columns produced by merge (e.g., Cells_x / Cells_y -> Cells)
def _coalesce_suffix(df, col):
    if col in df.columns:
        return df
    x, y = f"{col}_x", f"{col}_y"
    if x in df.columns or y in df.columns:
        left = df[x] if x in df.columns else pd.Series(index=df.index, dtype=object)
        right = df[y] if y in df.columns else pd.Series(index=df.index, dtype=object)
        df[col] = left.where(~left.isna(), right)
        for c in (x, y):
            if c in df.columns:
                df.drop(columns=c, inplace=True)
    return df

for col in META_COLS:
    coarrest_with_meta = _coalesce_suffix(coarrest_with_meta, col)

# Sanity checks
if coarrest_with_meta.empty:
    raise RuntimeError("Merged table is empty. Check that 'File_name' matches between summary and metadata.")
for col in ['Cells','Treatment','Condition']:
    if col not in coarrest_with_meta.columns:
        raise RuntimeError(f"Column '{col}' not found after merge. "
                           f"Available columns: {list(coarrest_with_meta.columns)}")

# Cast to string categories and make the composite Cells_Treatment
for col in ['Cells', 'Treatment', 'Condition']:
    coarrest_with_meta[col] = coarrest_with_meta[col].astype(str)
coarrest_with_meta['Cells_Treatment'] = coarrest_with_meta['Cells'] + "_" + coarrest_with_meta['Treatment']

# Save merged table (optional; helpful to inspect once)
per_fov_out = os.path.join(OUT_DIR, "co_arrest_by_FOV_with_metadata.csv")
coarrest_with_meta.to_csv(per_fov_out, index=False)
print("Saved:", per_fov_out)

# ---- Helper to plot and save a boxplot+strip for a grouping column ----
def plot_box(group_col: str, label: str, fname_stub: str):
    plt.figure(figsize=(max(10, min(18, 1.2*coarrest_with_meta[group_col].nunique())), 6))
    sns.boxplot(data=coarrest_with_meta, x=group_col, y='frac_together')
    sns.stripplot(data=coarrest_with_meta, x=group_col, y='frac_together',
                  color='black', alpha=0.5, jitter=True)
    plt.ylabel('Co-arrest fraction (per FOV)')
    plt.title(f'Co-arrest fraction by {label} (FOV-level){_title_suffix()}')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    out_pdf = os.path.join(OUT_DIR, f"boxplot_frac_together_by_{fname_stub}.pdf")
    plt.savefig(out_pdf)
    plt.close()
    print("Saved:", out_pdf)

# ---- Make all four plots ----
plot_box('Cells', 'Cells', 'Cells')
plot_box('Condition', 'Condition', 'Condition')
plot_box('Treatment', 'Treatment', 'Treatment')
plot_box('Cells_Treatment', 'Cells_Treatment', 'Cells_Treatment')


## **3.4 Identification of the number of cells in each co-arrested cluster**


In [ ]:
# @title ## Co-arrest group size per event (space–time components; r = k × diameter)

import os
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

# Reuse: OUT_DIR_CO (CoArrest_TimeWindow), DELTA_T_UNITS, K_RADIUS, meta_by_fov_cells (with diam_um, r_fov_um)
# from the previous cell. If not present, rebuild meta_by_fov_cells as in Patch A.

# --- Helper: Union-Find for connected components ---
class UnionFind:
    def __init__(self, n):
        self.parent = np.arange(n)
        self.rank = np.zeros(n, dtype=int)
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra == rb: return
        if self.rank[ra] < self.rank[rb]:
            self.parent[ra] = rb
        elif self.rank[rb] < self.rank[ra]:
            self.parent[rb] = ra
        else:
            self.parent[rb] = ra
            self.rank[ra] += 1

def label_group_sizes_for_fov(ev_fov: pd.DataFrame, r: float, dt: float) -> pd.DataFrame:
    ev = ev_fov.reset_index(drop=True).copy()
    n = len(ev)
    if n == 0:
        return ev.assign(
            co_arrest_neighbors=pd.Series(dtype=int),
            co_arrest_group_id=pd.Series(dtype=int),
            co_arrest_group_size=pd.Series(dtype=int)
        )
    X = ev[['x0','y0']].to_numpy(float)
    T = ev['t0'].to_numpy(float)
    D = cdist(X, X)
    dT = np.abs(T[:, None] - T[None, :])
    A = (D <= r) & (dT <= dt)
    np.fill_diagonal(A, False)

    neighbors = A.sum(axis=1).astype(int)

    uf = UnionFind(n)
    ii, jj = np.where(A)
    for a, b in zip(ii, jj):
        uf.union(a, b)
    roots = np.array([uf.find(i) for i in range(n)])
    _, comp_ids = np.unique(roots, return_inverse=True)
    comp_sizes = pd.Series(comp_ids).value_counts().to_dict()
    group_size = np.array([comp_sizes[c] for c in comp_ids], dtype=int)

    ev['co_arrest_neighbors'] = neighbors
    ev['co_arrest_group_id'] = comp_ids
    ev['co_arrest_group_size'] = group_size
    ev['co_arrest_together'] = ev['co_arrest_group_size'] >= 2
    return ev

# ---- Load the slowdown events ----
EVENTS_DIR = os.path.join(Results_Folder, "Slowdown_TimeColored")
all_events_path = os.path.join(EVENTS_DIR, "slowdown_events_all.csv")
if not os.path.exists(all_events_path):
    raise RuntimeError("No slowdown events found. Run the slowdown export first.")
events_df = pd.read_csv(all_events_path)

required = {'File_name','t0','x0','y0'}
missing = required - set(events_df.columns)
if missing:
    raise ValueError(f"Missing columns in events table: {missing}")

# ---- Label per-event group sizes for all FOVs using per-FOV r_fov ----
r_map = dict(zip(meta_by_fov_cells['File_name'], meta_by_fov_cells['r_fov_um']))
labeled = []
for fov, g in events_df.groupby('File_name'):
    r_fov = float(r_map[fov])
    lab = label_group_sizes_for_fov(g, r_fov, DELTA_T_UNITS)
    lab["r_fov_um"] = r_fov
    lab["k_radius"] = K_RADIUS
    lab["delta_t_units"] = DELTA_T_UNITS
    labeled.append(lab)

labeled_events = pd.concat(labeled, ignore_index=True)

# ---- Save annotated per-event table & per-FOV summary ----
TAG = f"rk{K_RADIUS}_dt{DELTA_T_UNITS}"
events_with_groups_path = os.path.join(OUT_DIR_CO, f"slowdown_events_with_groups_{TAG}.csv")
labeled_events.to_csv(events_with_groups_path, index=False)

per_fov_rows = []
for fov, g in labeled_events.groupby('File_name'):
    total = len(g)
    together_events = int((g['co_arrest_group_size'] >= 2).sum())
    alone_events = total - together_events
    size_counts = g['co_arrest_group_size'].value_counts().sort_index()

    row = {
        "File_name": fov,
        "r_fov_um": float(r_map[fov]),
        "k_radius": K_RADIUS,
        "delta_t_units": DELTA_T_UNITS,
        "total_events": total,
        "together_events": together_events,
        "alone_events": alone_events,
        "frac_together": together_events / total if total else np.nan,
        "mean_group_size": float(g['co_arrest_group_size'].mean()) if total else np.nan,
        "median_group_size": float(g['co_arrest_group_size'].median()) if total else np.nan,
        "max_group_size": int(g['co_arrest_group_size'].max()) if total else np.nan,
        "mean_direct_neighbors": float(g['co_arrest_neighbors'].mean()) if total else np.nan,
        "median_direct_neighbors": float(g['co_arrest_neighbors'].median()) if total else np.nan,
    }
    for k, v in size_counts.items():
        row[f"count_size_{int(k)}"] = int(v)
        row[f"frac_size_{int(k)}"] = float(v) / total if total else np.nan
    per_fov_rows.append(row)

summary_groups_df = pd.DataFrame(per_fov_rows).sort_values("File_name")
summary_groups_path = os.path.join(OUT_DIR_CO, f"co_arrest_group_summary_by_FOV_{TAG}.csv")
summary_groups_df.to_csv(summary_groups_path, index=False)

print("Saved per-event groups to:", events_with_groups_path)
print("Saved per-FOV group summary to:", summary_groups_path)


In [ ]:
# @title ## QC: Co-arrest flag vs component size agreement
import os
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

# -------- Resolve file paths (prefer variables from earlier cells) --------
try:
    flags_path = events_path  # from Co-arrest fraction cell
except NameError:
    co_dir = os.path.join(Results_Folder, "CoArrest_TimeWindow")
    cand = [fn for fn in os.listdir(co_dir)
            if fn.startswith("slowdown_events_with_coarrestflags_") and fn.endswith(".csv")]
    if not cand:
        raise RuntimeError("Couldn't find slowdown_events_with_coarrestflags_*.csv in CoArrest_TimeWindow.")
    cand.sort()
    flags_path = os.path.join(co_dir, cand[-1])

try:
    groups_path = events_with_groups_path  # from group-size cell
except NameError:
    co_dir = os.path.join(Results_Folder, "CoArrest_TimeWindow")
    cand = [fn for fn in os.listdir(co_dir)
            if fn.startswith("slowdown_events_with_groups_") and fn.endswith(".csv")]
    if not cand:
        raise RuntimeError("Couldn't find slowdown_events_with_groups_*.csv in CoArrest_TimeWindow.")
    cand.sort()
    groups_path = os.path.join(co_dir, cand[-1])

print("[QC] flags:", flags_path)
print("[QC] groups:", groups_path)

ev_flags  = pd.read_csv(flags_path)
ev_groups = pd.read_csv(groups_path)

# -------- Merge keys sanity --------
keys = ['File_name','Unique_ID','t0','x0','y0']
missing_flags  = [k for k in keys if k not in ev_flags.columns]
missing_groups = [k for k in keys if k not in ev_groups.columns]
if missing_flags or missing_groups:
    raise ValueError(f"Missing merge keys. flags missing {missing_flags}; groups missing {missing_groups}")

# -------- If 'co_arrest_together' is missing, recompute it per FOV --------
def _co_arrest_flags_for_fov(ev_fov: pd.DataFrame, r: float, dt: float) -> np.ndarray:
    n = len(ev_fov)
    if n <= 1:
        return np.zeros(n, dtype=bool)
    X = ev_fov[['x0','y0']].to_numpy(float)
    T = ev_fov['t0'].to_numpy(float)
    D = cdist(X, X)
    dT = np.abs(T[:, None] - T[None, :])
    A = (D <= r) & (dT <= dt)
    np.fill_diagonal(A, False)
    return A.any(axis=1)

if 'co_arrest_together' not in ev_flags.columns:
    print("⚠️ 'co_arrest_together' not found in flags file — recomputing from positions and times.")

    # ΔT: prefer from file, then globals, else fallback
    if 'delta_t_units' in ev_flags.columns and pd.notna(ev_flags['delta_t_units'].iloc[0]):
        dt_in = float(ev_flags['delta_t_units'].iloc[0])
    elif 'DELTA_T_UNITS' in globals():
        dt_in = float(DELTA_T_UNITS)
    else:
        dt_in = 0.04
        print(f"   Using fallback ΔT={dt_in}")

    # r per FOV:
    # 1) If r_fov_um is in the file, use it
    if 'r_fov_um' in ev_flags.columns:
        r_map = ev_flags.groupby('File_name')['r_fov_um'].first().to_dict()
    # 2) Else if meta_by_fov_cells exists (built earlier), use it
    elif 'meta_by_fov_cells' in globals() and 'r_fov_um' in meta_by_fov_cells.columns:
        r_map = dict(zip(meta_by_fov_cells['File_name'], meta_by_fov_cells['r_fov_um']))
    # 3) Else recompute from DIAMETER_MAP_UM & K_RADIUS
    else:
        if ('DIAMETER_MAP_UM' not in globals()) or ('K_RADIUS' not in globals()):
            raise RuntimeError("Cannot rebuild radius: need r_fov_um in file OR meta_by_fov_cells "
                               "OR (DIAMETER_MAP_UM and K_RADIUS) defined.")
        if 'merged_tracks_df' not in globals():
            raise RuntimeError("Need merged_tracks_df to map Cells→diameter per FOV.")
        meta_tmp = (
            merged_tracks_df.groupby('File_name', as_index=False)[['Cells']]
            .agg(lambda s: s.iloc[0])
        )
        meta_tmp['Cells'] = meta_tmp['Cells'].astype(str)
        unknown = sorted(set(meta_tmp['Cells'].unique()) - set(DIAMETER_MAP_UM.keys()))
        if unknown:
            raise ValueError(f"Add diameters for these Cells labels in DIAMETER_MAP_UM: {unknown}")
        meta_tmp['diam_um'] = meta_tmp['Cells'].map(DIAMETER_MAP_UM).astype(float)
        meta_tmp['r_fov_um'] = float(K_RADIUS) * meta_tmp['diam_um']
        r_map = dict(zip(meta_tmp['File_name'], meta_tmp['r_fov_um']))

    # Compute flags per FOV and insert column
    flags_series = []
    for fov, g in ev_flags.groupby('File_name'):
        r_fov = float(r_map[fov])
        flags = _co_arrest_flags_for_fov(g, r_fov, dt_in)
        flags_series.append(pd.Series(flags, index=g.index))
    ev_flags['co_arrest_together'] = pd.concat(flags_series).reindex(ev_flags.index).astype(bool)

# -------- Merge with groups and check agreement --------
merged = ev_flags.merge(
    ev_groups[keys + ['co_arrest_group_size']],
    on=keys,
    how='inner',
    validate='one_to_one'
)

merged['together_by_group'] = merged['co_arrest_group_size'] >= 2
mismatch = (merged['co_arrest_together'] != merged['together_by_group'])

n = len(merged)
n_mismatch = int(mismatch.sum())
print(f"[QC] Compared {n} events; mismatches: {n_mismatch}")

if n_mismatch:
    try:
        display(merged.loc[mismatch].head(10))
    except NameError:
        print(merged.loc[mismatch].head(10))
else:
    print("✅ Co-arrest classification matches component size (size ≥ 2) for all events.")

# Optional: per-FOV localization
per_fov_ok = (
    merged.assign(eq=(merged['co_arrest_together'] == merged['together_by_group']))
          .groupby('File_name')['eq'].all()
)
if not per_fov_ok.all():
    bad = per_fov_ok[~per_fov_ok].index.tolist()
    print(f"⚠️ FOVs with mismatches: {bad}")


In [ ]:
# @title ## Event-level co-arrest group size by Cells / Condition / Treatment / Cells_Treatment (clusters only)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- Load per-event groups (prefer in-memory; else from saved paths; else latest on disk) ----
if 'labeled_events' in globals():
    ev = labeled_events.copy()
else:
    try:
        # from Cell 5 if available
        ev = pd.read_csv(events_with_groups_path)
        print(f"[Event-level] Loaded from events_with_groups_path: {events_with_groups_path}")
    except Exception:
        co_dir = os.path.join(Results_Folder, "CoArrest_TimeWindow")
        cand = [fn for fn in os.listdir(co_dir)
                if fn.startswith("slowdown_events_with_groups_") and fn.endswith(".csv")]
        if not cand:
            raise RuntimeError("No slowdown_events_with_groups_*.csv found. Run the group-size cell first.")
        cand.sort()
        ev_path = os.path.join(co_dir, cand[-1])
        ev = pd.read_csv(ev_path)
        print(f"[Event-level] Loaded latest: {ev_path}")

# ---- Attach metadata to events ----
meta_cols = ['Cells','Condition','Treatment']
meta_by_fov = (
    merged_tracks_df
    .groupby('File_name', as_index=False)[meta_cols]
    .agg(lambda s: s.iloc[0])
)
for c in meta_cols:
    meta_by_fov[c] = meta_by_fov[c].astype(str)

evm = ev.merge(meta_by_fov, on='File_name', how='left')
evm['Cells_Treatment'] = evm['Cells'] + "_" + evm['Treatment']

# ---- Buckets: clusters only (size ≥2). If you want singles included, see note below. ----
def bucket_size(k: int) -> str:
    try:
        k = int(k)
    except:
        return "NA"
    return str(k) if k <= 5 else "6+"

evm = evm[evm['co_arrest_group_size'] >= 2].copy()
evm['size_bucket'] = evm['co_arrest_group_size'].apply(bucket_size)
bucket_order = ['2','3','4','5','6+']

# ---- Output folder ----
OUT_DIR_EVT = os.path.join(Results_Folder, "CoArrest_TimeWindow_EventLevel_GroupSize")
os.makedirs(OUT_DIR_EVT, exist_ok=True)

# Pull k and ΔT for titles if present
k_in  = (evm['k_radius'].iloc[0] if 'k_radius' in evm.columns else (K_RADIUS if 'K_RADIUS' in globals() else None))
dt_in = (evm['delta_t_units'].iloc[0] if 'delta_t_units' in evm.columns else (DELTA_T_UNITS if 'DELTA_T_UNITS' in globals() else None))
def _title_suffix():
    bits = []
    if k_in is not None:  bits.append(f"k={k_in:g}")
    if dt_in is not None: bits.append(f"ΔT={dt_in}")
    return f" ({', '.join(bits)})" if bits else ""

def stacked_by(group_col: str, label: str):
    # Counts & fractions per size bucket
    tab = (
        evm.groupby([group_col, 'size_bucket'])
           .size()
           .unstack('size_bucket', fill_value=0)
           .reindex(columns=bucket_order, fill_value=0)
           .sort_index()
    )
    frac = tab.div(tab.sum(axis=1).replace(0, np.nan), axis=0)

    # Save tables
    tab.to_csv(os.path.join(OUT_DIR_EVT, f"event_counts_cluster_sizes_by_{label}.csv"))
    frac.to_csv(os.path.join(OUT_DIR_EVT, f"event_fractions_cluster_sizes_by_{label}.csv"))

    # Stacked bar (PDF)
    fig, ax = plt.subplots(figsize=(12,6))
    bottom = np.zeros(len(frac))
    x = np.arange(len(frac.index))
    for b in bucket_order:
        vals = frac[b].values if b in frac.columns else np.zeros(len(frac))
        ax.bar(x, vals, bottom=bottom, label=b)
        bottom += np.nan_to_num(vals)
    ax.set_xticks(x)
    ax.set_xticklabels(frac.index, rotation=45, ha='right')
    ax.set_ylabel('Fraction of events (clusters only)')
    ax.set_xlabel(label)
    ax.set_title(f'Event-level co-arrest cluster-size distribution by {label}{_title_suffix()}')
    ax.legend(title='Cluster size', bbox_to_anchor=(1.02,1), loc='upper left')
    fig.tight_layout()
    out_pdf = os.path.join(OUT_DIR_EVT, f"stacked_event_cluster_sizes_by_{label}.pdf")
    fig.savefig(out_pdf)
    plt.close(fig)
    print("Saved:", out_pdf)

# ---- Make all four sets ----
stacked_by('Cells', 'Cells')
stacked_by('Condition', 'Condition')
stacked_by('Treatment', 'Treatment')
stacked_by('Cells_Treatment', 'Cells_Treatment')
